Check assigned GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct 18 01:30:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Check assigned memory

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


Download Object Detection API repo

In [ ]:
! rm -rf object-detection-api && git clone https://github.com/CatchZeng/object-detection-api.git

Cloning into 'object-detection-api'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 165 (delta 53), reused 151 (delta 39), pack-reused 0
Receiving objects: 100% (165/165), 1.69 MiB | 19.61 MiB/s, done.
Resolving deltas: 100% (53/53), done.


Install Object Detection API

In [ ]:
! cd object-detection-api && make install

if [ -d "./models" ]; then \
    echo 'models downloaded'; \
  else \
        git clone --depth=1 https://github.com/tensorflow/models; \
fi
Cloning into 'models'...
remote: Enumerating objects: 2959, done.
remote: Counting objects: 100% (2959/2959), done.
remote: Compressing objects: 100% (2490/2490), done.
remote: Total 2959 (delta 752), reused 1266 (delta 429), pack-reused 0
Receiving objects: 100% (2959/2959), 33.04 MiB | 32.57 MiB/s, done.
Resolving deltas: 100% (752/752), done.
cd models/research && \
protoc object_detection/protos/*.proto --python_out=. && \
cp object_detection/packages/tf2/setup.py . && \
python -m pip install .
Processing /content/object-detection-api/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this 

Download pre-trained Mask R-CNN model

In [ ]:
! cd object-detection-api/workspace/test-mask/ && make dl-model

mkdir -p pre-trained-models; \
model=mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8; \
curl -O http://download.tensorflow.org/models/object_detection/tf2/20200711/$model.tar.gz; \
tar zxvf $model.tar.gz; \
mv -f $model ./pre-trained-models/; \
rm -rf $model $model.tar.gz;
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  462M  100  462M    0     0   129M      0  0:00:03  0:00:03 --:--:--  129M
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/checkpoint
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/

Train model

In [ ]:
! cd object-detection-api/workspace/test-mask && make train

python model_main_tf2.py \
--model_dir=models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8 \
--pipeline_config_path=models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
2021-10-18 01:40:14.364386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 01:40:14.594210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 01:40:14.595279: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 01:40:14.600703: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative va

Test model

In [ ]:
! cd object-detection-api/workspace/test-mask && make export && python test_images.py

python exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config \
--trained_checkpoint_dir models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/ \
--output_directory exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8
2021-10-18 04:35:52.334203: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 04:35:52.389430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 04:35:52.390437: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-1

Download models

In [ ]:
from google.colab import files

!zip -r /content/exported-modles.zip /content/object-detection-api/workspace/test-mask/exported-models
files.download("/content/exported-modles.zip")

  adding: content/object-detection-api/workspace/test-mask/exported-models/ (stored 0%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/ (stored 0%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config (deflated 70%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/ (stored 0%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/assets/ (stored 0%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/saved_model.pb (deflated 92%)
  adding: content/object-detection-api/workspace/test-mask/exported-models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/s

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>